# Fetching crypto and tweets data.

In [132]:
from sklearn.preprocessing import OrdinalEncoder
from crypto_api import CryptoApi
import pandas as pd
import numpy as np
import re



In [12]:
from TweetScraper import TweetScraper
import pandas as pd


scrp = TweetScraper(start='2023-05-04', end='2023-05-14', maxEmptyPages=2, max_workers=8)
new_tweets = scrp.parallel_download_tweets()

Could not translate t.co card URL on tweet 1641596848921276417
Stopping after 2 empty pages
Stopping after 2 empty pages
Stopping after 2 empty pages
Stopping after 2 empty pages
Stopping after 2 empty pages
Stopping after 2 empty pages
Stopping after 2 empty pages
Stopping after 2 empty pages
Stopping after 2 empty pages
Stopping after 2 empty pages


In [13]:
pd.DataFrame(new_tweets)

,url,date,rawContent,renderedContent,id,user,replyCount,retweetCount,likeCount,quoteCount,...,mentionedUsers,coordinates,place,hashtags,cashtags,card,viewCount,vibe,bookmarkCount,pinned
0,https://twitter.com/elonmusk/status/1654258632...,2023-05-04 22:56:07+00:00,@DanFriedman81 @NateSilver538 That’s being fixed,@DanFriedman81 @NateSilver538 That’s being fixed,1654258632316387329,"{'username': 'elonmusk', 'id': 44196397, 'disp...",115,60,1178,7,...,"[{'username': 'DanFriedman81', 'id': 178353631...",None,None,None,None,None,69358,None,6,None
1,https://twitter.com/elonmusk/status/1654258447...,2023-05-04 22:55:23+00:00,@NateSilver538 Great!,@NateSilver538 Great!,1654258447443959808,"{'username': 'elonmusk', 'id': 44196397, 'disp...",184,127,2680,8,...,"[{'username': 'NateSilver538', 'id': 16017475,...",None,None,None,None,None,482878,None,10,None
2,https://twitter.com/elonmusk/status/1654243799...,2023-05-04 21:57:10+00:00,@Gfilche @OpenAI An excellent question,@Gfilche @OpenAI An excellent question,1654243799936167936,"{'username': 'elonmusk', 'id': 44196397, 'disp...",360,224,3620,15,...,"[{'username': 'Gfilche', 'id': 348844712, 'dis...",None,None,None,None,None,542352,None,28,None
3,https://twitter.com/elonmusk/status/1654243462...,2023-05-04 21:55:50+00:00,@GRDecter !,@GRDecter !,1654243462567321608,"{'username': 'elonmusk', 'id': 44196397, 'disp...",320,250,3344,14,...,"[{'username': 'GRDecter', 'id': 12814572675821...",None,None,None,None,None,543078,None,11,None
4,https://twitter.com/elonmusk/status/1654240480...,2023-05-04 21:43:59+00:00,@zerohedge This is a *major* issue!,@zerohedge This is a *major* issue!,1654240480882749441,"{'username': 'elonmusk', 'id': 44196397, 'disp...",711,985,10084,55,...,"[{'username': 'zerohedge', 'id': 18856867, 'di...",None,None,None,None,None,2168624,None,85,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
437,https://twitter.com/elonmusk/status/1656824770...,2023-05-12 00:53:02+00:00,@johnkrausphotos @chazman Should be much better,@johnkrausphotos @chazman Should be much better,1656824770292506625,"{'username': 'elonmusk', 'id': 44196397, 'disp...",90,88,1574,7,...,"[{'username': 'johnkrausphotos', 'id': 3072160...",None,None,None,None,None,88018,None,4,None
438,https://twitter.com/elonmusk/status/1656824520...,2023-05-12 00:52:02+00:00,@CollinRugg I only ever see cnn on this platfo...,@CollinRugg I only ever see cnn on this platfo...,1656824520752394240,"{'username': 'elonmusk', 'id': 44196397, 'disp...",1728,2045,39748,115,...,"[{'username': 'CollinRugg', 'id': 890061634181...",None,None,None,None,None,1736097,None,43,None
439,https://twitter.com/elonmusk/status/1656823602...,2023-05-12 00:48:23+00:00,@WallStreetSilv Close tho,@WallStreetSilv Close tho,1656823602736680960,"{'username': 'elonmusk', 'id': 44196397, 'disp...",547,356,8780,19,...,"[{'username': 'WallStreetSilv', 'id': 13665656...",None,None,None,None,None,705610,None,27,None
440,https://twitter.com/elonmusk/status/1656823160...,2023-05-12 00:46:38+00:00,@PeterDiamandis Good point,@PeterDiamandis Good point,1656823160799625216,"{'username': 'elonmusk', 'id': 44196397, 'disp...",180,121,2128,13,...,"[{'username': 'PeterDiamandis', 'id': 14710129...",None,None,None,None,None,309182,None,11,None


# Analysis of tweets

In [59]:
'''
TODO: GENERAL DATA TASKS:
0) find a way to deal with multiple tweets for a day
1) merge 2 datasets into 
2) imput missing data, maybe try interpolation or expectation maximization
    2.1) compare with mean, median imput methods
3) ivestigate relationship within data, maybe correlation matrix etc
'''

tweets_df = pd.read_csv(r'Data/elon_tweets.csv', index_col=0)
tweets_df['date'] = pd.to_datetime(tweets_df['date'])

btc_df = pd.read_csv('Data/btc_data.csv')
btc_df['time'] = pd.to_datetime(btc_df['time'])

tweets_df = tweets_df[tweets_df['date'].dt.date.isin(btc_df['time'].dt.date)]
                                               
# tweets_df = (tweets_df
#              .dropna(axis=1, how='all')
#              .drop(['vibe','cashtags'], axis=1)) # 1 and 18 notna values respectively  

## Dealing with sparse columns

In [134]:
sparse_cols = tweets_df.columns[tweets_df.notnull().mean() < 1.0].values.copy()

mod_tweets_df = tweets_df.copy()
mod_tweets_df = (mod_tweets_df[mod_tweets_df['lang']=='en']
                 .drop(['id','url','source','sourceUrl'], axis=1)                 
                 .reset_index(drop=True)
                 .copy())

mod_tweets_df = mod_tweets_df.drop(['lang'], axis=1)

## Data cleaning and preprocessing

In [135]:
encoder = OrdinalEncoder()
mod_tweets_df['sourceLabel_encoded'] = encoder.fit_transform(mod_tweets_df['sourceLabel'].values.reshape(-1, 1))
mod_tweets_df['isReplied']   = [0 if type(tweet)==float else 1 for tweet in mod_tweets_df['inReplyToUser']]
mod_tweets_df['isMentioned'] = [0 if type(tweet)==float else 1 for tweet in mod_tweets_df['mentionedUsers']]

#mod_tweets_df = mod_tweets_df.drop(['sourceLabel','inReplyToUser','mentionedUsers'], axis=1)


def extract_dict(line: str, prepare_to_df: False):
    """Extracts data from a dict represented as string and makes it a dict.

    ## Parameters:
        line (str): row of a Series/DataFrame to be preprocessed.
        prepare_to_df (bool): prepares extracted dict to be wrapped into DataFrame.

    ## Returns:
        dict: extracted dict from string.
    """    

    extracted_content = dict(re.findall(r"'(\w+)': '?({.*}|datetime.datetime\(.*\)|[\w\d/:\. ]*)'?", line))
    
    # Wraps dict values into lists to be easily represented as a DataFrame row.
    if prepare_to_df:
        for key,value in extracted_content.items():
            if value == '':
                extracted_content[key] = [None]
            else:
                extracted_content[key] = value
        
    return extracted_content


new_df = mod_tweets_df.copy()     
extracted_df = (pd.DataFrame([*mod_tweets_df['user']
                              .apply(lambda x: extract_dict(x, True))])
                )
# TODO: try this json approach, but replace datetime via regex to str and then upcast to datetime again
# my_json = my_json.replace("'",'"').replace('None','null')
# json.loads(my_json)

new_df = (pd.concat([new_df, extracted_df], axis=1)
            .drop(['user','username','id','displayname','verified','created',
                    'location','protected','profileImageUrl','profileBannerUrl',
                    'rawDescription','renderedDescription','favouritesCount',
                    'friendsCount','mediaCount','statusesCount'], axis=1))

In [136]:
# Converting columns containing numbers to int after extraction.
for column in new_df:
    if 'Count' in column:
        new_df[column] = new_df[column].astype('Int64').copy()

In [137]:
new_df[['rawContent','isReplied','isMentioned']].query("rawContent.str.contains('@')")

new_df['mentionsCount'] = new_df['rawContent'].str.count(r'@[\w\d]+')
new_df['mentions'] = new_df['rawContent'].apply(lambda x : re.findall(r'(@[^\s]+)', x))

count = 0
for a,b in new_df[['mentionsCount','mentions']].values:
    if a==len(b):
        count +=1 
print(count==len(new_df))

new_df['charCount'] = new_df['rawContent'].apply(lambda x: len(x))
new_df = new_df.drop('descriptionLinks', axis=1)

True


In [138]:
#links = tweets_df[tweets_df.columns[tweets_df.columns.isin(new_df.columns)==False]]['links'].value_counts().copy()

tweets_df[tweets_df['rawContent']=='True']['quotedTweet'].iloc[0]

'{\'url\': \'https://twitter.com/ggreenwald/status/1625871270737809408\', \'date\': datetime.datetime(2023, 2, 15, 14, 54, 52, tzinfo=datetime.timezone.utc), \'rawContent\': "The corporate media\'s ability to -- overnight -- turn anyone who dissents in anyway into some sort of fascist or even Hitler-like figure, and then have millions of their followers go around mindlessly repeating it, is both impressive and chilling:", \'renderedContent\': "The corporate media\'s ability to -- overnight -- turn anyone who dissents in anyway into some sort of fascist or even Hitler-like figure, and then have millions of their followers go around mindlessly repeating it, is both impressive and chilling:", \'id\': 1625871270737809408, \'user\': {\'username\': \'ggreenwald\', \'id\': 16076032, \'displayname\': \'Glenn Greenwald\', \'rawDescription\': \'Journalist; Author; Host, @SystemUpdate_; Columnist, @Folha; Co-Founder: The Intercept, @TheInterceptBr; @abrigo_hope, @FreedomofPress, @ongcriadefavela.

In [139]:
import re
import json
import datetime


def clean_text(raw_text):    
    cleaned_text = re.sub(r' \'?(displayname|renderedDescription)\'?: (.*?)(\'|None),', '', raw_text)
    cleaned_text = (cleaned_text
                    .replace("'",'"')
                    .replace('None','null')
                    .replace('True','true')
                    .replace('False','false'))
    # cleaned_text = re.sub(r'(\w+)"(\w+)', r"\1'\2", cleaned_text)
    
    return cleaned_text


def deserialize(text):    
    deserialized_texts = []
    extract_dicts = re.findall(r'{.*?}',text)
    
    for str_dict in extract_dicts:
        cleaned_text = clean_text(str_dict)

        pattern = r'datetime.datetime\(.*\)'
        cleaned_text = re.sub(f'({pattern})',r'"\1"',cleaned_text)
        
        deserialized_text = json.loads(cleaned_text)
        
        if deserialized_text['created']!=None:
            deserialized_text['created'] = eval(deserialized_text['created'])
        
        deserialized_texts.append(deserialized_text)

    return deserialized_texts
    
    
new_df = new_df.drop('inReplyToUser',axis=1)
new_df['mentionedUsers'] = new_df['mentionedUsers'].apply(lambda x: deserialize(x) if type(x)==str else None)

In [144]:
# new_df['sourceLabel'] encode this 1,2,3
new_df = new_df.drop(['links','media','link'], axis=1)

for item in tweets_df.columns:
    print(f"{item:<18} {item in new_df['quotedTweet'][new_df['quotedTweet'].notna()].iloc[0]}")

url                True
date               True
rawContent         True
renderedContent    True
id                 True
user               True
replyCount         True
retweetCount       True
likeCount          True
quoteCount         True
conversationId     True
lang               True
source             True
sourceUrl          True
sourceLabel        True
links              True
media              True
quotedTweet        True
inReplyToTweetId   True
inReplyToUser      True
mentionedUsers     True
hashtags           True
card               True
viewCount          True


In [209]:
object_cols = [col for col in new_df if new_df[col].dtype==object]
print(object_cols)

# new_df['hashtags'][new_df['hashtags'].notna()]

['rawContent', 'renderedContent', 'sourceLabel', 'quotedTweet', 'mentionedUsers', 'hashtags', 'card', 'label', 'mentions', 'vectorized', 'TopicsProbs']


In [218]:
new_df[new_df['quotedTweet'].notna() & new_df['card'].notna()].iloc[0,]

date                                           2019-05-04 23:05:47+00:00
rawContent             Order online in 2 mins, 7 day return policy ht...
renderedContent        Order online in 2 mins, 7 day return policy Te...
replyCount                                                           777
retweetCount                                                        1780
likeCount                                                          22999
quoteCount                                                           177
conversationId                                       1124812395300343808
sourceLabel                                           Twitter for iPhone
quotedTweet            {'url': 'https://twitter.com/cleantechnica/sta...
inReplyToTweetId                                                     NaN
mentionedUsers                                                      None
hashtags                                                             NaN
card                   {'title': 'Electric Cars, So

In [145]:
tweets_df['quotedTweet'][tweets_df['quotedTweet'].notna()].iloc[0]

'{\'url\': \'https://twitter.com/SpaceX/status/614551591601844224\', \'date\': datetime.datetime(2015, 6, 26, 21, 51, 31, tzinfo=datetime.timezone.utc), \'rawContent\': \'Rocket completed hold down firing today in advance of Sunday cargo mission to @Space_Station http://t.co/tdni53IviI http://t.co/PjiYSIDTOu\', \'renderedContent\': \'Rocket completed hold down firing today in advance of Sunday cargo mission to @Space_Station spacex.com/webcast http://t.co/PjiYSIDTOu\', \'id\': 614551591601844224, \'user\': {\'username\': \'SpaceX\', \'id\': 34743251, \'displayname\': \'SpaceX\', \'rawDescription\': \'SpaceX designs, manufactures and launches the world’s most advanced rockets and spacecraft\', \'renderedDescription\': \'SpaceX designs, manufactures and launches the world’s most advanced rockets and spacecraft\', \'descriptionLinks\': None, \'verified\': True, \'created\': datetime.datetime(2009, 4, 23, 21, 53, 30, tzinfo=datetime.timezone.utc), \'followersCount\': 28768901, \'friendsCou

In [57]:
# ok fuck it, let's fuse something from new_df to crypto data

btc_df = pd.read_csv('Data/btc_data.csv')
btc_df['time'] = pd.to_datetime(btc_df['time'])

tweets_df[tweets_df['date'].dt.date.isin(btc_df['time'].dt.date)]

,url,date,rawContent,renderedContent,id,user,replyCount,retweetCount,likeCount,quoteCount,...,inReplyToTweetId,inReplyToUser,mentionedUsers,coordinates,place,hashtags,cashtags,card,viewCount,vibe
2932,https://twitter.com/elonmusk/status/9308768459...,2017-11-15 19:15:09+00:00,"Sorry, date is tomorrow night at 8pm. This pic...","Sorry, date is tomorrow night at 8pm. This pic...",930876845959946240,"{'username': 'elonmusk', 'id': 44196397, 'disp...",683,695,12536,65,...,9.308757e+17,"{'username': 'elonmusk', 'id': 44196397, 'disp...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2933,https://twitter.com/elonmusk/status/9308757393...,2017-11-15 19:10:45+00:00,"It can transform into a robot, fight aliens an...","It can transform into a robot, fight aliens an...",930875739397791744,"{'username': 'elonmusk', 'id': 44196397, 'disp...",1935,12438,64591,1542,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2934,https://twitter.com/elonmusk/status/9306817600...,2017-11-15 06:19:57+00:00,@rjcc @Tesla Do you really believe that someon...,@rjcc @Tesla Do you really believe that someon...,930681760026853377,"{'username': 'elonmusk', 'id': 44196397, 'disp...",136,62,854,19,...,9.306783e+17,"{'username': 'rjcc', 'id': 15009761, 'displayn...","[{'username': 'rjcc', 'id': 15009761, 'display...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
2935,https://twitter.com/elonmusk/status/9306776291...,2017-11-15 06:03:32+00:00,@rjcc @Tesla That is a *counterpoint* to illus...,@rjcc @Tesla That is a *counterpoint* to illus...,930677629161758726,"{'username': 'elonmusk', 'id': 44196397, 'disp...",36,27,711,5,...,9.306759e+17,"{'username': 'rjcc', 'id': 15009761, 'displayn...","[{'username': 'rjcc', 'id': 15009761, 'display...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
2936,https://twitter.com/elonmusk/status/9314120200...,2017-11-17 06:41:45+00:00,Tesla Semi https://t.co/7VLz7F46Ji,Tesla Semi https://t.co/7VLz7F46Ji,931412020016029696,"{'username': 'elonmusk', 'id': 44196397, 'disp...",1879,20114,103777,2398,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21770,https://twitter.com/elonmusk/status/1644862856...,2023-04-09 00:40:39+00:00,@teslaownersSV Got to break a few eggs to make...,@teslaownersSV Got to break a few eggs to make...,1644862856448339969,"{'username': 'elonmusk', 'id': 44196397, 'disp...",4826,4900,113346,391,...,1.644817e+18,"{'username': 'teslaownersSV', 'id': 1016059981...","[{'username': 'teslaownersSV', 'id': 101605998...",NaN,NaN,NaN,NaN,NaN,5883818.0,NaN
21771,https://twitter.com/elonmusk/status/1644861079...,2023-04-09 00:33:35+00:00,@Jason @DeanPreston @GrowSF Good question,@Jason @DeanPreston @GrowSF Good question,1644861079107895298,"{'username': 'elonmusk', 'id': 44196397, 'disp...",166,179,2663,14,...,1.644820e+18,"{'username': 'Jason', 'id': 3840, 'displayname...","[{'username': 'Jason', 'id': 3840, 'displaynam...",NaN,NaN,NaN,NaN,NaN,518864.0,NaN
21772,https://twitter.com/elonmusk/status/1644860793...,2023-04-09 00:32:27+00:00,"Friendship takes work, enmity is effortless","Friendship takes work, enmity is effortless",1644860793224085506,"{'username': 'elonmusk', 'id': 44196397, 'disp...",8821,20876,145759,1571,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15961060.0,NaN
21773,https://twitter.com/elonmusk/status/1644857727...,2023-04-09 00:20:16+00:00,@BillyM2k 🤣,@BillyM2k 🤣,1644857727166955523,"{'username': 'elonmusk', 'id': 44196397, 'disp...",261,231,4476,14,...,1.644818e+18,"{'username': 'BillyM2k', 'id': 30699048, 'disp...","[{'username': 'BillyM2k', 'id': 30699048, 'dis...",NaN,NaN,NaN,NaN,NaN,656404.0,NaN


In [14]:
tweets_df

NameError: name 'tweets_df' is not defined

# NLP Model Bulding

## Preprocessing and vectorizing

In [146]:
from sklearn.feature_extraction.text import TfidfVectorizer
from gensim import corpora, matutils, utils, models
import spacy


def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])
    url_pattern = [{"label": "URL",
                    "pattern": [{"LIKE_URL": True}]}]

    ruler = nlp.add_pipe('entity_ruler', before='ner')
    ruler.add_patterns(url_pattern)
    
    texts_out = []
    if type(texts)!=list:
        texts = [texts]
    
    for text in texts:
        # TODO: consider using nlp.pipe which should be faster
        doc = nlp(text)
        cleaned_text = []
        for token in doc:
            if token.ent_type_ != 'URL' and not token.is_stop and token.pos_ in allowed_postags:
                cleaned_text.append(token.lemma_)
        final = ' '.join(cleaned_text)
        texts_out.append(final)

    return texts_out


def create_ngrams(texts):
    data_words = []
    for text in texts:
        new = utils.simple_preprocess(text)
        data_words.append(new)

    bigrams_phrases  = models.Phrases(data_words, min_count=3, threshold=50)
    trigrams_phrases = models.Phrases(bigrams_phrases[data_words], threshold=50)

    bigram  = models.phrases.Phraser(bigrams_phrases)
    trigram = models.phrases.Phraser(trigrams_phrases)

    data_bigrams = [bigram[doc] for doc in data_words]
    data_bigrams_trigrams = [trigram[bigram[doc]] for doc in data_bigrams]
    
    return data_bigrams_trigrams


def vectorize_texts(texts_ngrams):
    id2word = corpora.Dictionary(texts_ngrams)
    corpus = [id2word.doc2bow(text) for text in texts_ngrams]
    return id2word, corpus


In [147]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer


def make_custom_pipeline(steps):
    for i, step in enumerate(steps):
        steps.insert(i, (step[0], FunctionTransformer(step[1])))
        steps.remove(step)

    return Pipeline(steps)


# with open('lemmatized_texts.txt', 'r', encoding="utf-8") as f:
#     lemmatized_texts = f.readlines()
    
# lemmatized_texts = [line.replace('\n','') for line in lemmatized_texts]

# lemmatized_texts = lemmatization(new_df['rawContent'])
steps = [('lemmatization', lemmatization),
         ('trigrams', create_ngrams),
         ('vectorization', vectorize_texts)]

preprocessing_pipeline = make_custom_pipeline(steps)
id2word, corpus = preprocessing_pipeline.transform(new_df['rawContent'].values.tolist())

# data_bigrams_trigrams = create_ngrams(lemmatized_texts)
# id2word, corpus = vectorize_texts(data_bigrams_trigrams)

#tfidf = models.TfidfModel(corpus, id2word=id2word)
# tfidf_vectorizer = TfidfVectorizer(max_df=0.6,
#                                    min_df=5,
#                                    ngram_range=(1,3))
# tfidf_matrix = tfidf_vectorizer.fit_transform(lemmatized_texts)

# id2word = dict((v, k) for k, v in tfidf_vectorizer.vocabulary_.items())
# corpus = matutils.Sparse2Corpus(tfidf_matrix.T)

In [148]:
# low_value = 0.03
# words = []
# words_missing_in_tfidf = []

# for i in range(0, len(corpus)):
#     bow = corpus[i]
#     low_value_words = []
#     tfidf_ids = [id for id,_ in tfidf[bow]]
#     bow_idf = [id for id,_ in bow]
#     low_value_words = [id for id, value in tfidf[bow] if value < low_value]
#     drops = low_value_words+words_missing_in_tfidf
    
#     for item in drops:
#         words.append(id2word[item])
    
#     # words with tfidf score of 0 will be missing
#     words_missing_in_tfidf = [id for id in bow_idf if id not in tfidf_ids]
#     new_bow = [b for b in bow if b[0] not in low_value_words and b[0] not in words_missing_in_tfidf]
#     corpus[i] = new_bow

In [149]:
from gensim.models import CoherenceModel


def bayesian_tuning(model, params_grid: dict, texts, verbose=False):
    # TODO: implement bayesian tuning
    models_scores = {}
    for i in range(95,170,5):
        lda_model = model(corpus=params_grid['corpus'],
                          num_topics=i,
                          id2word=params_grid['id2word'],
                          random_state=1,
                          passes=10,
                          per_word_topics=True)
        
        coherence_model_lda = CoherenceModel(model=lda_model, 
                                             texts=texts, 
                                             corpus=params_grid['corpus'], 
                                             dictionary=params_grid['id2word']
                                             )
        coherence_score = coherence_model_lda.get_coherence()
        
        models_scores.update({lda_model: coherence_score})
        
        if verbose:
            print(f'Topics {i:<3}: {coherence_score}')
    
    return models_scores


params_grid = {'corpus':corpus,  
               'num_topics':10, 
               'id2word':id2word, 
               'random_state':1, 
               'update_every':1, 
               'chunksize':3000, 
               'passes':2}

# lda_models_scores = bayesian_tuning(LdaMulticore, params_grid, texts, verbose=True)

In [150]:
import pyLDAvis.gensim_models as gensimvis
from gensim.models import LdaMulticore
import pyLDAvis


pyLDAvis.enable_notebook()

lda_model = LdaMulticore(corpus=corpus,
                         num_topics=15,
                         id2word=id2word,
                         random_state=1,
                         passes=10,
                         per_word_topics=True)

vis = gensimvis.prepare(lda_model, corpus, id2word)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
7     -0.092898 -0.042626       1        1  9.026206
9     -0.033240  0.080919       2        1  8.931155
4     -0.080176 -0.056455       3        1  7.426712
14    -0.106723 -0.033665       4        1  7.169323
0     -0.018483  0.038885       5        1  6.976951
11    -0.141309 -0.049458       6        1  6.956383
8      0.004386  0.041588       7        1  6.734716
1     -0.034478  0.069864       8        1  6.657594
5      0.100507  0.032057       9        1  6.575940
2      0.022076  0.055656      10        1  6.154911
10     0.024160  0.013155      11        1  5.957681
3      0.071576  0.027278      12        1  5.729896
13     0.116251 -0.197288      13        1  5.390875
6      0.090145  0.101358      14        1  5.199554
12     0.078206 -0.081269      15        1  5.112102, topic_info=         Term        Freq       Total Category  logprob  loglift
999   exactly  393.000000  393.000000  Default  30.0000  30.0000
29       true  334.000000  334.000000  Default  29.0000  29.0000
699     thank  190.000000  190.000000  Default  28.0000  28.0000
263      look  260.000000  260.000000  Default  27.0000  27.0000
70       good  803.000000  803.000000  Default  26.0000  26.0000
...       ...         ...         ...      ...      ...      ...
9232   billym   22.249530  450.160431  Topic15  -5.4792  -0.0337
200     right   21.357492  320.529300  Topic15  -5.5201   0.2650
697       big   18.275864  229.900692  Topic15  -5.6759   0.4415
763       use   17.751941  201.761295  Topic15  -5.7050   0.5430
6        need   17.590433  546.570109  Topic15  -5.7142  -0.4627

[1161 rows x 6 columns], token_table=       Topic      Freq         Term
term                               
9639       9  0.898079        abide
611        3  0.928460        abort
186        2  0.041403   absolutely
186        5  0.890165   absolutely
186        8  0.006901   absolutely
...      ...       ...          ...
10011      3  0.719557         yike
10011     14  0.179889         yike
11355      6  0.831986  zaleskiluke
10753      4  0.915714    zerohedge
10885      7  0.861646    zubymusic

[4065 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[8, 10, 5, 15, 1, 12, 9, 2, 6, 3, 11, 4, 14, 7, 13])

In [151]:
def prepare_new_texts(texts: list[str]):
    steps = [('lemmatization', lemmatization),
            ('trigrams', create_ngrams)]

    corpus_pipeline = make_custom_pipeline(steps)
    texts_ngrams = corpus_pipeline.transform(texts)
    corpus = [id2word.doc2bow(text) for text in texts_ngrams]
    
    return corpus

In [152]:
# corpus = [id2word.doc2bow(text) for text in texts_ngrams]
# lda_model.get_document_topics(corpus, minimum_probability=0)

In [153]:
new_df['vectorized']  = preprocessing_pipeline.transform(new_df['rawContent'].values.tolist())[1]
new_df['TopicsProbs'] = new_df['vectorized'].apply(lambda x: dict(lda_model.get_document_topics(x, minimum_probability=0)))

In [154]:
new_df['TopicsProbs']

0        {0: 0.011117808, 1: 0.844351, 2: 0.011117779, ...
1        {0: 0.01334675, 1: 0.013346679, 2: 0.013346687...
2        {0: 0.0060679186, 1: 0.006067915, 2: 0.0060679...
3        {0: 0.016689455, 1: 0.76634765, 2: 0.016689451...
4        {0: 0.0067009046, 1: 0.0067009293, 2: 0.006700...
                               ...                        
18311    {0: 0.0066674403, 1: 0.006667429, 2: 0.0066674...
18312    {0: 0.006668687, 1: 0.0066686873, 2: 0.0066686...
18313    {0: 0.013338832, 1: 0.01333883, 2: 0.013338835...
18314    {0: 0.022222506, 1: 0.022222506, 2: 0.02222252...
18315    {0: 0.84435683, 1: 0.011117369, 2: 0.011117374...
Name: TopicsProbs, Length: 18316, dtype: object

In [155]:
btc_df = pd.read_csv('Data/btc_data.csv')

In [156]:
btc_df.iloc[0:1, 1:-2]

,time,high,low,open,volumefrom,volumeto,close
0,2017-11-15,7330.06,6596.94,6597.06,131120.23,9.229595e+08,7283.22


In [157]:
btc_df

,Unnamed: 0,time,high,low,open,volumefrom,volumeto,close,conversionType,conversionSymbol
0,0,2017-11-15,7330.06,6596.94,6597.06,131120.23,9.229595e+08,7283.22,direct,NaN
1,1,2017-11-16,7964.64,7119.17,7283.02,133937.80,1.010131e+09,7853.68,direct,NaN
2,2,2017-11-17,8000.19,7534.70,7853.68,117347.03,9.175999e+08,7699.95,direct,NaN
3,3,2017-11-18,7857.52,7458.90,7699.95,74382.12,5.711211e+08,7780.91,direct,NaN
4,4,2017-11-19,8100.87,7675.41,7781.02,68618.85,5.413913e+08,8042.64,direct,NaN
...,...,...,...,...,...,...,...,...,...,...
1996,1996,2023-05-04,29370.41,28703.83,29041.00,27555.47,7.986150e+08,28866.54,direct,NaN
1997,1997,2023-05-05,29697.23,28853.10,28866.54,35357.40,1.037025e+09,29550.84,direct,NaN
1998,1998,2023-05-06,29857.86,28459.77,29550.84,24511.97,7.131925e+08,28943.94,direct,NaN
1999,1999,2023-05-07,29207.72,28547.14,28943.94,16591.33,4.802422e+08,28574.43,direct,NaN


In [158]:
datesCount = new_df.groupby(new_df['date'].dt.date)['date'].count()
new_df = new_df.merge(datesCount.rename('tweetsCount'), left_on=new_df['date'].dt.date, right_index=True, how='left')

In [159]:
plot_this = new_df.groupby(new_df['date'].dt.date).count()['date']
plot_this

date
2010-06-04     1
2011-12-01     2
2011-12-03     2
2011-12-04     1
2011-12-21     7
              ..
2023-04-06    22
2023-04-07    21
2023-04-08    22
2023-04-09    47
2023-04-10    44
Name: date, Length: 2655, dtype: int64